In [116]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
import optuna

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [117]:
# Load datasets
train_data=pd.read_csv( '/content/train_dataset.csv')
test_data =pd.read_csv('/content/test_dataset.csv')


In [118]:
# Fill missing values explicitly
train_data['Arrival Delay in Minutes'] = train_data['Arrival Delay in Minutes'].fillna(train_data['Arrival Delay in Minutes'].median())
test_data['Arrival Delay in Minutes'] = test_data['Arrival Delay in Minutes'].fillna(test_data['Arrival Delay in Minutes'].median())

In [119]:
# Encode categorical variables
categorical_features = ['Gender', 'Customer Type', 'Type of Travel', 'Class']
encoder = LabelEncoder()
for feature in categorical_features:
    train_data[feature] = encoder.fit_transform(train_data[feature])
    test_data[feature] = encoder.transform(test_data[feature])

In [120]:
# Define features and target
X = train_data.drop(['id', 'satisfaction'], axis=1)
y = train_data['satisfaction']
X_test = test_data.drop(['id'], axis=1)

In [121]:
# Split the training data for validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [122]:
# Scale numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [123]:
# Objective function for Optuna
def objective(trial):
    # Define hyperparameter search space
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 300, step=50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, step=0.01),
        "max_depth": trial.suggest_int("max_depth", 3, 7, step=2),
        "subsample": trial.suggest_float("subsample", 0.8, 1.0, step=0.1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.8, 1.0, step=0.1),
        "random_state": 42
    }

    # Train model with the current set of hyperparameters
    model = XGBClassifier(**param, use_label_encoder=False, eval_metric="logloss")
    model.fit(X_train_split, y_train_split)

    # Evaluate on the validation set
    preds = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, preds)
    return accuracy

In [124]:
# Run Optuna optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

[I 2024-12-27 06:17:00,226] A new study created in memory with name: no-name-72aa81ca-559f-4210-879b-772ce8316521
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:17:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-12-27 06:17:00,472] Trial 0 finished with value: 0.954375 and parameters: {'n_estimators': 150, 'learning_rate': 0.060000000000000005, 'max_depth': 7, 'subsample': 1.0, 'colsample_bytree': 0.8}. Best is trial 0 with value: 0.954375.
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:17:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-12-27 06:17:00,593] Trial 1 finished with value: 0.9325 and parameters: {'n_estimators': 150, 'learning_rate': 0.02, 'max_depth': 3, 'subsample': 1.0, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.954375

In [125]:
# Print the best parameters and accuracy
print("Best Parameters:", study.best_params)
print("Best Validation Accuracy:", study.best_value)

Best Parameters: {'n_estimators': 250, 'learning_rate': 0.05, 'max_depth': 5, 'subsample': 0.8, 'colsample_bytree': 0.9}
Best Validation Accuracy: 0.956875


In [126]:
# Train the final model with the best hyperparameters
best_params = study.best_params
final_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric="logloss")
final_model.fit(X_train, y_train)

# Generate predictions on the test set (if available)
if 'X_test' in globals():
    test_preds = final_model.predict(X_test)
    submission = pd.DataFrame({'id': test_data['id'], 'satisfation': test_preds})

    # Save predictions to CSV
    submission.to_csv('submission.csv', index=False)
    print("Submission saved as 'submission.csv'")
else:
    print("Test set not provided, cannot generate submission.")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:17:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Submission saved as 'submission.csv'
